In [6]:
import numpy as np
import pandas as pd

import quantutils.dataset.pipeline as ppl
import quantutils.dataset.ml as mlutils
from marketinsights.api.model import MarketInsightsModel
from marketinsights.remote.ml import MIAssembly
from marketinsights.remote.models import MIModelServer

In [8]:
DATASET_ID1 = "4234f0f1b6fcc17f6458696a6cdf5101"  # DOW
MODEL_ID = "3a491b1a-8af6-416d-aa14-f812cbd660bb"

modelSvr = MIModelServer(secret="marketinsights-k8s-cred")
assembly = MIAssembly(modelSvr, secret="marketinsights-k8s-cred")
modelConfig = assembly.get_model(MODEL_ID)

# Training Set
dataset, descriptor = assembly.get_dataset_by_id(DATASET_ID1, debug=False)

# Test Set
dataset2 = pd.read_pickle("testset.pkl")
dataset

,0,1,2,3,4,5,6,7,8,9
Date_Time,,,,,,,,,,
2013-01-02 15:00:00-05:00,0.390661,0.826167,0.199723,0.508553,0.503930,1.000000,0.000000,0.629219,0.162306,1
2013-01-03 15:00:00-05:00,0.766138,1.000000,0.585742,0.943056,0.943056,0.943925,0.000000,0.486633,0.342863,0
2013-01-04 15:00:00-05:00,0.096931,1.000000,0.000000,0.631664,0.632741,0.905223,0.340334,0.847604,0.138530,1
2013-01-07 15:00:00-05:00,0.078826,0.468008,0.000000,0.337731,0.337401,1.000000,0.253628,0.984499,0.227052,1
2013-01-08 15:00:00-05:00,0.000000,0.747823,0.000000,0.685475,0.684779,1.000000,0.344479,0.518635,0.215921,1
...,...,...,...,...,...,...,...,...,...,...
2018-07-26 15:00:00-04:00,0.117647,0.779412,0.000000,0.500000,0.492647,1.000000,0.345588,0.654412,0.266526,0
2018-07-27 15:00:00-04:00,0.377660,0.888298,0.000000,0.861702,0.845745,1.000000,0.287234,0.489362,0.370020,1
2018-07-30 15:00:00-04:00,0.306818,0.954545,0.000000,0.892045,0.903409,1.000000,0.369318,0.482955,0.347277,0


In [5]:
mlmodel = MarketInsightsModel.createInstance(
    "Perceptron", 
    opts={"name": "MLModel", "featureCount": descriptor["features"], "labelCount": descriptor["labels"], "modelConfig": modelConfig }
)

In [6]:
train_x, train_y = ppl.splitCol(dataset, descriptor["features"])
test_x, test_y = ppl.splitCol(dataset2, descriptor["features"])

In [7]:
mlutils.evaluate(ppl.onehot(mlmodel.predictions(train_x).numpy()), ppl.onehot(train_y), threshold=0.5)

Won : 709.0
Lost : 668.0
Total : 1377.0
Diff : 41.0
Edge : 2.9774872912127814%
IR : 1.1048845


0.51488745

In [8]:
mlmodel.restore("../config/modelserver/mlmodel/3")

In [9]:
mlutils.evaluate(ppl.onehot(mlmodel.predictions(train_x).numpy()), ppl.onehot(train_y), threshold=0.5)

Won : 1033.0
Lost : 344.0
Total : 1377.0
Diff : 689.0
Edge : 50.03631082062454%
IR : 18.56745


0.75018156

In [11]:
mlutils.evaluate(ppl.onehot(mlmodel.predictions(test_x).numpy()), ppl.onehot(test_y), threshold=0.8)

Won : 33.0
Lost : 17.0
Total : 50.0
Diff : 16.0
Edge : 32.0%
IR : 2.2627416


0.66

In [14]:
import tensorflow as tf
train_x = tf.cast(train_x, tf.float32)
train_y = tf.cast(train_y, tf.float32)
mlmodel.train(train_x, train_y, learning_rate=3.0, epochs=100)

Mean squared error for step 0: 0.481
Precision 0.75


In [15]:
predictions = assembly.get_predictions_with_dataset_id(dataset_id=DATASET_ID1, training_run_id="mlmodel")
predictions

,0,1,2,3,4,5,6,7,8,y_pred0
Date_Time,,,,,,,,,,
2013-01-02 15:00:00-05:00,0.390661,0.826167,0.199723,0.508553,0.503930,1.000000,0.000000,0.629219,0.162306,0.567616
2013-01-03 15:00:00-05:00,0.766138,1.000000,0.585742,0.943056,0.943056,0.943925,0.000000,0.486633,0.342863,0.453158
2013-01-04 15:00:00-05:00,0.096931,1.000000,0.000000,0.631664,0.632741,0.905223,0.340334,0.847604,0.138530,0.561873
2013-01-07 15:00:00-05:00,0.078826,0.468008,0.000000,0.337731,0.337401,1.000000,0.253628,0.984499,0.227052,1.000000
2013-01-08 15:00:00-05:00,0.000000,0.747823,0.000000,0.685475,0.684779,1.000000,0.344479,0.518635,0.215921,0.550138
...,...,...,...,...,...,...,...,...,...,...
2018-07-26 15:00:00-04:00,0.117647,0.779412,0.000000,0.500000,0.492647,1.000000,0.345588,0.654412,0.266526,0.575855
2018-07-27 15:00:00-04:00,0.377660,0.888298,0.000000,0.861702,0.845745,1.000000,0.287234,0.489362,0.370020,0.574508
2018-07-30 15:00:00-04:00,0.306818,0.954545,0.000000,0.892045,0.903409,1.000000,0.369318,0.482955,0.347277,0.543834


In [17]:
mlutils.evaluate(ppl.onehot(predictions[["y_pred0"]].values), ppl.onehot(train_y.numpy()), threshold=0.5)

Won : 1033.0
Lost : 344.0
Total : 1377.0
Diff : 689.0
Edge : 50.03631082062454%
IR : 18.56745


0.75018156